In [92]:
from numpy import genfromtxt
import os
import pandas as pd
import numpy as np
import gc

In [162]:
gc.collect()
survey_path = os.path.join(os.path.abspath(''),'surveys')
surveys = {}
for j,survey in enumerate(os.listdir(survey_path)):
    if 'Participant' in survey:
        continue
    print(survey)
    user_id = survey[5]
    survey_file = os.path.join(survey_path,survey)
    df = pd.read_csv(survey_file,dtype='str')
    experiments = []
    for i,sl in enumerate([slice(6,17),slice(18,29), slice(30,41), slice(43,54), slice(55,66), slice(67,78)]):
        exp_df = df.iloc[sl]
        exp_df = exp_df.rename(columns={x:y for x,y in zip(df.columns,range(0,len(df.columns)))})
        exp_df = exp_df.fillna(0)
        exp_data = [exp_df.iloc[0][0], exp_df.iloc[1:-2].set_index(0), exp_df.iloc[-2:]]
        exp_data[1] = exp_data[1].replace('x',1)
        exp_data[1] = exp_data[1].iloc[:,list(range(7))]
        exp_data[1] = exp_data[1].astype(int)
        exp_data[1] = exp_data[1].dot(range(1,8))
        exp_data[1] = pd.DataFrame(exp_data[1]).transpose()
        experiments.append(exp_data)

    exp,ratings,comments = zip(*experiments)
    rate_df = pd.concat(ratings)
    rate_df = ratings_df.reset_index(drop=True)
    rate_df_ls, rate_df_b = rate_df[:3],rate_df[3:]
    rate_df_ls = pd.DataFrame(rate_df_ls.reset_index(drop=True))
    rate_df_b = pd.DataFrame(rate_df_b.reset_index(drop=True))
    surveys[user_id] = {
                    'experiment names': exp,
                    'all ratings': ratings,
                    'lightswitch ratings': rate_df_ls,
                    'bottle ratings': rate_df_b,
                    'comments': comments
                        }

User 0 - Sheet1.csv
User 1 - Sheet1.csv
User 10 - Sheet1.csv
User 2 - Sheet1.csv
User 3 - Sheet1.csv
User 4 - Sheet1.csv
User 5 - Sheet1.csv
User 6 - Sheet1.csv
User 7 - Sheet1.csv
User 9 - Sheet1.csv


In [163]:
survey_file = os.path.join(survey_path,'Participant Info - Sheet1.csv')
df = pd.read_csv(survey_file,dtype='str')
df = df[1:13][['User ID','Condition']]
df = df.set_index('User ID')
user_cond = df.to_dict()['Condition']
letter_number = {'A': 1, 'B': 2, 'C': 3}
for k,v in user_cond.items():
    order = list(v)
    order = [letter_number[letter] for letter in order]
    user_cond[k] = order

In [164]:
for k in surveys.keys():
    surveys[k]['lightswitch ratings'] = surveys[k]['lightswitch ratings'].reindex(user_cond[k])
    surveys[k]['bottle ratings'] = surveys[k]['bottle ratings'].reindex(user_cond[k])

In [165]:
print(surveys['0']['lightswitch ratings'])

{'experiment names': ('After Phase 1 - Light Switch', 'After Phase 2 - Light Switch', 'After Phase 3 - Light Switch', 'After Phase 1 - Bottle Maze', 'After Phase 2 - Bottle Maze', 'After Phase 3 - Bottle Maze'), 'all ratings': (0  The system performed the task I wanted  I felt in control  \
0                                       2                  2   

0  The system responded to my input in the way that I expected  \
0                                                  2             

0  The system was competent at performing tasks, even if they weren't the tasks I wanted  \
0                                                  6                                       

0  The system improved over time  I improved at using the system over time  \
0                              1                                         1   

0  I always looked directly at my final target, holding the same gaze throughout an episode  \
0                                                  7                     